# GRiD SOFTWARE v5
---
## Objectives
- Make the bot move to a desired aruco in a straight line and stop at the centre of aruco.
- Given N bots with N list of paths, Guide the bots sequentially. (N is variable)
- Exhaustive Error Handling and Scalable Soultion
---
## PipeLine
- Send a 32 bit number to ESP, comprising RPM of 4 wheels : DONE
- Figure out the mathematics of mapping wheel RPMs to Bot movement
- Implement Proportional Control for navigation : DONE
- Construct proper Loops to handle all cases

## Import Necessary Modules

In [1]:
import socket
import cv2

## Constants Used

In [2]:
port = 8093
url = 'Insert Cam URL here'
p = 60
bots = [] #Enter the Bot IDs in sequence
path = [[]] #Enter the path of each bot in sequence
flip = [] #Enter aruco_IDS to flip
max_d = 0
max_ang = 3.14
rpm = [0,0,0,0] #fr,fl,br,bl

## Functions Used

In [3]:
def distCalc(r1, r2):
    x1, y1 = r1
    x2, y2 = r2 
    
    return math.dist((x1,y1), (x2,y2))

def centreCalc(r1, r2):
    return (int((r1[0]+r2[0])/2), int((r1[1]+r2[1])/2))

def angleCalc(r1, r2):
    x1, y1 = r1
    x2, y2 = r2
    
    inner_prod = x1*x2 + y1*y2
    len1 = math.hypot(x1, y1)
    len2 = math.hypot(x2, y2)
    
    angle = math.acos(inner_prod/(len1*len2))
    
    if (x1*y2 - x2*y1) < 0:
        angle = -angle
    
    return angle

## Server

In [4]:
def getHostIP():
    hostname = socket.gethostname()
    my_ip = socket.gethostbyname(hostname)
    print(my_ip)

def startServer(port):
    global serversocket
    # INET is for IPv4 and STREAM is for TCP
    serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  
    host = '0.0.0.0'

    try:
        serversocket.bind((host,port))
    except socket.error as e:
        print(str(e))

    print("Waiting for connection")
    
    serversocket.listen(2)

    global client, addr
    client, addr = serversocket.accept()
    print("Connection made with " + str(client))

def closeServer():
    client.close()
    serversocket.close()
    print("Server closed successfully")
    
def sendRPM(rpm):
    data = '-'.join(str(x) for x in rpm)
    data += '-'
    print(data)
    client.sendto(data.encode('UTF-8'), addr)
    #if sent == 0:
     #   print("SOCKET Connection broken")
        
def sendCommand(x):
    client.sendto(x.encode('UTF-8'), addr)

## Save the Video

In [5]:
def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

    #for sorting the file names properly
    files = sorted(files, key=lambda x: int(x.split('.')[1]))

    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each files
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        #print(filename)
        #inserting the frames into an image array
        frame_array.append(img)

    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()
    shutil.rmtree('./data')
    

#convert_frames_to_video('./data/', 'tracker_22_09_21_1.mp4', 10.0)

## Bot Tracker

In [6]:
def arucoDetector(img) :
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    key = getattr(aruco, f'DICT_4X4_250')
    arucoDict = aruco.Dictionary_get(key)
    arucoParam = aruco.DetectorParameters_create()

    bbox, ids, rejected = aruco.detectMarkers(imgGray, arucoDict, parameters=arucoParam)
    
    img = aruco.drawDetectedMarkers(img, bbox)
    
    return img, bbox, ids

## Parameters Calculator

In [7]:
def arucoOrient(img, bbox, ids, bot, dest = 0) :
    bot_flag = 0
    dest_flag = 0
    for i in range(len(ids)):
        if ids[i]==dest:
            dest_coords = centreCalc(bbox[i][0][0],bbox[i][0][2])
            dest_flag = 1
        if ids[i]==bot_ids[bot]:
            bot_coords = centreCalc(bbox[i][0][0], bbox[i][0][2])
            bot_front = centreCalc(bbox[i][0][0], bbox[i][0][1])
            bot_rear = centreCalc(bbox[i][0][2], bbox[i][0][3])
            bot_flag = 1


    img = cv2.arrowedLine(img, bot_rear, bot_front, (0,0,255), 2)
    img = cv2.arrowedLine(img, bot_coords, dest_coords, (0,255,0), 2)
    angle = angleCalc(np.subtract(np.array(bot_front), np.array(bot_rear)),np.subtract(np.array(dest_coords), np.array(bot_coords)))
    distance = distCalc(bot_coords, dest_coords)
    return img, bot_coords, dest_coords, distance, angle

## Navigation Algorithm

In [8]:
def commandfunc(distance, angle, r_thr = 10, theta_thr = 0.01):
    global max_d, max_ang, rpm
    flag=0
    if distance > max_d:
        max_d = distance
        
    
    #if angle > max_ang:
    #    max_ang = angle
    
       
    
    if abs(angle) < theta_thr :
        if distance > r_thr:
            v_lin = (distance/max_d)*255
            rpm = [v_lin,v_lin,v_lin,v_lin]
        else:
            rpm = [0,0,0,0]
            flag = 1
            max_d = 0
        
    elif angle > theta_thr :
        v_ang = (abs(angle)/max_ang)*255
        rpm = [0,v_ang,0,v_ang]
    
    elif angle < (-1*theta_thr) :
        v_ang = (abs(angle)/max_ang)*255
        rpm = [v_ang,0,v_ang,0]
    
    #print(max_d, max_ang)
    
    return rpm, flag

## Main Function

In [9]:
startServer(8091)

sendCommand('9')




Waiting for connection
Connection made with <socket.socket fd=512, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.137.1', 8091), raddr=('192.168.137.163', 61069)>


In [10]:
#sendCommand('1')

In [11]:
#rpm = [100]*4
#sendRPM(rpm)

In [12]:
#closeServer()